In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Load the data
df1 = pd.read_csv("muaBanNhaDat.csv")
df1.head()

,title,price,address,price_m2,rooms,toilets,direction,floors,property_legal_document,house_type,furnishing_sell,width,length,living_size,size,pty_characteristics
0,Bán nhà chính chủ Khu phố Tây - phường Phạm Ng...,"6,1 tỷ","14/21, Đường Ðỗ Quang Ðẩu, Phường Phạm Ngũ Lão...","203,33 triệu/m²",4 phòng,4 phòng,Đông Nam,3.00,Đã có sổ,"Nhà ngõ, hẻm",Bàn giao thô,NaN,NaN,NaN,NaN,NaN
1,"Bán nhà Chính chủ đường Hưng Phú, P9, Q8, DT 4...","3,5 tỷ","528A/15A , Đường Hưng Phú, Phường 9, Quận 8, T...","78,65 triệu/m²",2 phòng,2 phòng,Đông Nam,1.00,Đã có sổ,"Nhà ngõ, hẻm",Nội thất đầy đủ,NaN,NaN,NaN,NaN,NaN
2,3 lầu (3x13=39m2 vuông vức) 5 phòng hẻm nhựa 6...,"8,3 tỷ","Đường Nhiêu Tứ, Phường 7, Quận Phú Nhuận, Tp H...","212,82 triệu/m²",5 phòng,4 phòng,Tây Nam,4.00,Đã có sổ,"Nhà ngõ, hẻm",Nội thất đầy đủ,NaN,NaN,NaN,NaN,NaN
3,BÁN NHÀ GẤP QUẬN BÌNH CHÁNH TÂN LIÊM XÃ PHONG ...,899 triệu,"Đường Tân Liêm, Xã Phong Phú, Huyện Bình Chánh...","10,45 triệu/m²",2 phòng,3 phòng,NaN,1.00,Đã có sổ,Nhà phố liền kề,Nội thất đầy đủ,5 m,NaN,NaN,NaN,NaN
4,CHÍNH CHỦ BÁN CHDV 24TR/TH - NGUYỄN TRỌNG TUYỂ...,"6,5 tỷ","Đường Nguyễn Trọng Tuyển, Phường 2, Quận Tân B...","144,44 triệu/m²",6 phòng,NaN,NaN,NaN,Đã có sổ,"Nhà ngõ, hẻm",NaN,7 m,NaN,NaN,45 m²,NaN


In [3]:
def convert_price(price_str):
    if not isinstance(price_str, str):
        # Handle cases where the value is not a string (e.g., NaN or numeric values)
        return np.nan
    try:
        if 'tỷ' in price_str:
            return float(price_str.replace(' tỷ', '').replace(',', '.')) * 1e9
        elif 'triệu' in price_str:
            return float(price_str.replace(' triệu', '').replace(',', '.')) * 1e6
        elif 'nghìn' in price_str:
            return float(price_str.replace(' nghìn', '').replace(',', '.')) * 1e3
        else:
            # Handle cases where the value is already numeric
            return float(price_str.replace(',', '.'))
    except ValueError:
        print(f"ValueError: Could not convert {price_str} to float.")
        return np.nan

In [5]:
import re
# Function to clean and convert room and toilet columns
def clean_rooms_toilets(value):
    if pd.isna(value):
        return 0
    value = str(value).lower()
    # Extract number if the string contains 'nhiều hơn'
    if 'nhiều hơn' in value:
        match = re.search(r'\d+', value)
        if match:
            return int(match.group(0))
    # Clean and convert the value
    return int(value.replace('phòng', '').strip())

In [8]:
# Load the data
df1 = pd.read_csv("muaBanNhaDat.csv")
df1.head()

df1['urgent'] = df1['title'].apply(lambda x: 'yes' if 'gấp' in x.lower() or 'ngộp' in x.lower() else 'no')

# Handle 'location' column if it exists
if 'address' in df1.columns:
    # Filter out rows where 'address' contains 'Cập nhật'
    df1 = df1[~df1['address'].str.contains('Cập nhật', case=False, na=False)]

    # Split the 'address' into multiple parts based on ', ' and take the last two columns (district and province)
    df1[['ward','district', 'province']] = df1['address'].apply(lambda x: pd.Series(x.split(', ')[-3:]))

    # Remove "Tp " from the beginning of the 'province' column
    df1['province'] = df1['province'].str.replace(r'^Tp\s+', '', regex=True)

    # Drop the 'address' column after splitting
    df1.drop(columns=['address'], inplace=True)

df1 = df1[df1['province'] == "Hồ Chí Minh"]

# Drop rows where price is "Giá thỏa thuận"
df1 = df1[df1['price'] != "Giá thỏa thuận"]

# Apply content to null value
df1['rooms'] = df1['rooms'].apply(clean_rooms_toilets)
df1['toilets'] = df1['toilets'].apply(clean_rooms_toilets)

df1['floors'] = df1['floors'].fillna(0)
df1['house_type'] = df1['house_type'].fillna('')
df1['pty_characteristics'] = df1['pty_characteristics'].fillna('')
df1['furnishing_sell'] = df1['furnishing_sell'].fillna('Hoàn thiện cơ bản')
df1['province'] = df1['province'].fillna('')

# Apply conversion functions
df1['price'] = df1['price'].astype(str).apply(convert_price)

# Ensure all values in 'price_m2' are strings before extracting the numeric part
df1['price_m2'] = df1['price_m2'].astype(str)

# Extract numeric value from 'price_m2', convert commas to periods, and convert to float
df1['price_m2'] = df1['price_m2'].str.extract(r'([\d,.]+)').replace(',', '.', regex=True).astype(float)

# Convert 'price' and 'price_m2' to numeric, if not already
df1['price'] = pd.to_numeric(df1['price'], errors='coerce')
df1['price_m2'] = pd.to_numeric(df1['price_m2'], errors='coerce')

# Ensure all values in 'size' are strings before removing 'm²' and converting to float
df1['size'] = df1['size'].astype(str)

# Remove 'm²' and any spaces from 'size', then convert to float
df1['size'] = df1['size'].str.replace(r'm²', '', regex=True).str.strip().astype(float)

# Nếu cột 'size' có giá trị NaN, tính toán lại giá trị cho nó
df1['size'] = df1['size'].fillna(df1['price'] / (df1['price_m2'] * 1000000))

# Làm tròn cột 'size' đến 2 chữ số thập phân
df1['size'] = df1['size'].round(2)

df1.drop(columns=['price_m2'], inplace=True)
df1.drop(columns=['property_legal_document'], inplace=True)
df1.drop(columns=['direction'], inplace=True)
df1.drop(columns=['width'], inplace=True)
df1.drop(columns=['length'], inplace=True)
df1.drop(columns=['living_size'], inplace=True)
df1.drop(columns=['province'], inplace=True)
df1.drop(columns=['title'], inplace=True)


# Display the updated DataFrame
df1.head(50)


,price,rooms,toilets,floors,house_type,furnishing_sell,size,pty_characteristics,status,ward,district
0,6100000000.00,4,4,3.00,"Nhà ngõ, hẻm",Bàn giao thô,30.00,,no,Phường Phạm Ngũ Lão,Quận 1
1,3500000000.00,2,2,1.00,"Nhà ngõ, hẻm",Nội thất đầy đủ,44.50,,no,Phường 9,Quận 8
2,8300000000.00,5,4,4.00,"Nhà ngõ, hẻm",Nội thất đầy đủ,39.00,,no,Phường 7,Quận Phú Nhuận
3,899000000.00,2,3,1.00,Nhà phố liền kề,Nội thất đầy đủ,86.03,,yes,Xã Phong Phú,Huyện Bình Chánh
4,6500000000.00,6,0,0.00,"Nhà ngõ, hẻm",Hoàn thiện cơ bản,45.00,,no,Phường 2,Quận Tân Bình
5,6200000000.00,3,0,3.00,"Nhà ngõ, hẻm",Nội thất cao cấp,64.00,,no,Phường Phước Long A (Quận 9 cũ),Thành phố Thủ Đức
6,12500000000.00,5,5,3.00,"Nhà ngõ, hẻm",Hoàn thiện cơ bản,160.01,,no,Phường Bình Trị Đông,Quận Bình Tân
7,7800000000.00,5,0,0.00,"Nhà ngõ, hẻm",Hoàn thiện cơ bản,90.00,,no,Phường 11,Quận Gò Vấp
8,12200000000.00,10,6,5.00,"Nhà ngõ, hẻm",Nội thất đầy đủ,90.00,,no,Phường 13,Quận Bình Thạnh
9,5750000000.00,4,5,5.00,"Nhà ngõ, hẻm",Nội thất cao cấp,56.00,,no,Phường Bình Hưng Hoà A,Quận Bình Tân


In [18]:
# Kiểm tra giá trị thiếu
print(df1.isnull().sum())
print(df1.shape)

price              0
rooms              0
toilets            0
floors             0
house_type         0
furnishing_sell    0
size               0
ward               0
district           0
dtype: int64
(2752, 9)


In [9]:
# Convert 'price' and 'size' columns to numeric, forcing errors to NaN
df1['price'] = pd.to_numeric(df1['price'], errors='coerce')
df1['size'] = pd.to_numeric(df1['size'], errors='coerce')

# Drop rows where 'price' or 'size' are NaN
df1 = df1.dropna(subset=['price', 'size'])

# Compute IQR for price and size
Q1 = df1[['price', 'size']].quantile(0.25)
Q3 = df1[['price', 'size']].quantile(0.75)
IQR = Q3 - Q1

# Identify outliers
outliers = (df1[['price', 'size']] < (Q1 - 1.5 * IQR)) | (df1[['price', 'size']] > (Q3 + 1.5 * IQR))

# Remove outliers
df1 = df1[~outliers.any(axis=1)]

# Print the shape of the cleaned DataFrame
print("Shape of cleaned DataFrame:")
print(df1.shape)


Shape of cleaned DataFrame:
(428, 11)


In [13]:
print(df1.describe())
df1.shape

               price   rooms  toilets  floors    size
count        2458.00 2458.00  2458.00 2458.00 2458.00
mean   5553318958.50    3.29     2.55    2.07   58.90
std    3400675435.07    1.60     1.74    2.78   26.77
min     200000000.00    1.00     0.00    0.00   10.00
25%    3000000000.00    2.00     1.00    0.00   40.00
50%    5000000000.00    3.00     2.00    2.00   56.00
75%    7300000000.00    4.00     4.00    3.00   75.28
max   15800000000.00   10.00     6.00   45.00  143.10


(2458, 9)

In [20]:
df1.to_csv('houseDataset.csv',encoding="utf-8-sig",index=False)

In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Load dataset
df = pd.read_csv('houseDataset.csv')

# Loại bỏ các ngoại lệ dựa trên IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

# Encode các đặc trưng phân loại
encoder = OneHotEncoder(sparse_output=False)
categorical_features = ['ward','district', 'house_type', 'furnishing_sell']
encoded_features = encoder.fit_transform(df[categorical_features])

# Tạo DataFrame cho các đặc trưng đã mã hóa
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Kết hợp các đặc trưng mã hóa với các đặc trưng số học
X = pd.concat([encoded_df, df[['size', 'rooms', 'toilets', 'floors']].reset_index(drop=True)], axis=1)
y = df['price']

# Chuẩn hóa các đặc trưng số học
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Chia tập dữ liệu thành training và testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Khởi tạo các mô hình
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

# Tạo danh sách để lưu trữ kết quả
results = []

# Huấn luyện và đánh giá từng mô hình
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    results.append({'Model': name, 'R2 Score': r2, 'MSE': mse})
    print(f"{name} - R² Score: {r2:.2f}, MSE: {mse:,.2f}")

# Chuyển kết quả thành DataFrame
results_df = pd.DataFrame(results)

# Hiển thị bảng kết quả
print("\nKết quả đánh giá các mô hình:")
print(results_df)

Linear Regression - R² Score: -263150155507695683108864.00, MSE: 2,681,217,157,422,436,460,110,530,483,211,144,815,181,824.00
Ridge Regression - R² Score: 0.71, MSE: 2,962,506,134,669,538,816.00


d:\Workspace python\RealEstatePredict\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.514e+20, tolerance: 1.523e+18
  model = cd_fast.enet_coordinate_descent(


Lasso Regression - R² Score: 0.71, MSE: 2,995,429,709,094,796,800.00
Random Forest - R² Score: 0.75, MSE: 2,510,294,026,751,468,032.00
Gradient Boosting - R² Score: 0.70, MSE: 3,020,096,720,239,643,648.00

Kết quả đánh giá các mô hình:
               Model                     R2 Score  \
0  Linear Regression -263150155507695683108864.00   
1   Ridge Regression                         0.71   
2   Lasso Regression                         0.71   
3      Random Forest                         0.75   
4  Gradient Boosting                         0.70   

                                             MSE  
0 2681217157422436460110530483211144815181824.00  
1                         2962506134669538816.00  
2                         2995429709094796800.00  
3                         2510294026751468032.00  
4                         3020096720239643648.00  


In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Load dataset
df = pd.read_csv('houseDataset.csv')

# Encode các đặc trưng phân loại
encoder = OneHotEncoder(sparse_output=False)
categorical_features = ['ward', 'district', 'house_type', 'furnishing_sell']
encoded_features = encoder.fit_transform(df[categorical_features])

# Tạo DataFrame cho các đặc trưng đã mã hóa
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Kết hợp các đặc trưng mã hóa với các đặc trưng số học
X = pd.concat([encoded_df, df[['size', 'rooms', 'toilets', 'floors']].reset_index(drop=True)], axis=1)
y = df['price']

# Chuẩn hóa các đặc trưng số học
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Chia tập dữ liệu thành training và testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình Random Forest
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)

# Dự đoán giá nhà mới
new_house = pd.DataFrame({
'ward': ['Phường 2'],
    'district': ['Quận 5'],
    'size': [52],
    'rooms': [3],
    'toilets': [2],
    'floors': [2],
    'house_type': ['Nhà ngõ, hẻm'],
    'furnishing_sell': ['Nội thất đầy đủ'],
})

# Encode dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'house_type', 'furnishing_sell']])
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá nhà mới bằng mô hình Random Forest
predicted_price = random_forest_model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Random Forest: {predicted_price[0]:,.0f} VND")


Giá nhà dự đoán bằng Random Forest: 11,216,300,000 VND


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

df = pd.read_csv('houseDataset.csv')

# Assuming df is already prepared
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['ward', 'district', 'house_type', 'furnishing_sell', 'status']])

# Combine encoded features with numeric features
X = np.hstack((encoded_features.toarray(), df[['size', 'rooms', 'toilets', 'floors']].values))
y = df['price']

# Train the model
model = LinearRegression()
model.fit(X, y)

# Create a DataFrame for the new house
new_house = pd.DataFrame({
'ward': ['Phường Hiệp Phú (Quận 9 cũ)'],
'district': ['Thành phố Thủ Đức'],
'size': [20],
'rooms': [0],
'toilets': [0],
'floors': [0],
'house_type': ['Nhà ngõ, hẻm'],
'furnishing_sell': ['Hoàn thiện cơ bản'],
'status': ['yes'],
})

# Encode the new house data
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'house_type', 'furnishing_sell', 'status']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá nhà mới bằng mô hình Random Forest
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Linear Regression: {predicted_price[0]:,.0f} VND")

Giá nhà dự đoán bằng Linear Regression: 24,231,936 VND


In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor

# Tải dữ liệu
df = pd.read_csv('houseDataset.csv')

# Giả định df đã được chuẩn bị
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['ward', 'district', 'house_type', 'furnishing_sell', 'status']])

# Kết hợp các đặc trưng đã mã hóa với các đặc trưng số học
X = np.hstack((encoded_features.toarray(), df[['size', 'rooms', 'toilets', 'floors']].values))
y = df['price']

# Huấn luyện mô hình Gradient Boosting
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X, y)

# Tạo DataFrame cho nhà mới
new_house = pd.DataFrame({
'ward': ['Phường 2'],
'district': ['Quận 5'],
'size': [52],
'rooms': [3],
'toilets': [2],
'floors': [2],
'house_type': ['Nhà ngõ, hẻm'],
'furnishing_sell': ['Nội thất đầy đủ'],
'status': ['yes'],
})

# Mã hóa dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'house_type', 'furnishing_sell', 'status']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá nhà mới bằng mô hình Gradient Boosting
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Gradient Boosting: {predicted_price[0]:,.0f} VND")


Giá nhà dự đoán bằng Gradient Boosting: 4,913,709,749 VND


In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge

# Tải dữ liệu
df = pd.read_csv('houseDataset.csv')

# Giả định df đã được chuẩn bị
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['ward', 'district', 'house_type', 'furnishing_sell']])

# Kết hợp các đặc trưng đã mã hóa với các đặc trưng số học
X = np.hstack((encoded_features.toarray(), df[['size', 'rooms', 'toilets', 'floors']].values))
y = df['price']

# Huấn luyện mô hình Ridge Regression
model = Ridge(alpha=1.0)  # alpha là tham số điều chỉnh
model.fit(X, y)

# Tạo DataFrame cho nhà mới
new_house = pd.DataFrame({
'ward': ['Phường 2'],
'district': ['Quận 5'],
'size': [52],
'rooms': [3],
'toilets': [2],
'floors': [2],
'house_type': ['Nhà ngõ, hẻm'],
'furnishing_sell': ['Nội thất đầy đủ'],
})

# Mã hóa dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'house_type', 'furnishing_sell']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá nhà mới bằng mô hình Ridge Regression
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Ridge Regression: {predicted_price[0]:,.0f} VND")


Giá nhà dự đoán bằng Ridge Regression: 6,848,288,506 VND


In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso

# Tải dữ liệu
df = pd.read_csv('houseDataset.csv')

# Giả định df đã được chuẩn bị
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['ward', 'district', 'house_type', 'furnishing_sell']])

# Kết hợp các đặc trưng đã mã hóa với các đặc trưng số học
X = np.hstack((encoded_features.toarray(), df[['size', 'rooms', 'toilets', 'floors']].values))
y = df['price']

# Huấn luyện mô hình Lasso Regression
model = Lasso(alpha=0.1)  # alpha là tham số điều chỉnh
model.fit(X, y)

# Tạo DataFrame cho nhà mới
new_house = pd.DataFrame({
'ward': ['Phường 2'],
'district': ['Quận 5'],
'size': [52],
'rooms': [3],
'toilets': [2],
'floors': [2],
'house_type': ['Nhà ngõ, hẻm'],
'furnishing_sell': ['Nội thất đầy đủ'],
})

# Mã hóa dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'house_type', 'furnishing_sell']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá nhà mới bằng mô hình Lasso Regression
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Lasso Regression: {predicted_price[0]:,.0f} VND")


Giá nhà dự đoán bằng Lasso Regression: 6,962,793,704 VND


d:\Workspace python\RealEstatePredict\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.443e+20, tolerance: 2.387e+18
  model = cd_fast.enet_coordinate_descent(
